# Tensorflow を使ったテキストデータの分類

## ニュース記事の分類器を作る
　学習データはLivedoorニュースコーパスを用います。

　Livedoor ニュースコーパスとは
- 本コーパスは、NHN Japan株式会社が運営する「Livedoor ニュース」のうち、下記のクリエイティブ・コモンズライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成したものです。
- 収集時期は2012年9月上旬
- 収集されているデータは、
  - ニューストピック
  - Sports Watch
  - ITライフハック
  - 家電チャンネル
  - MOVIE ENTER
  - 独女通信
  - エスマックス
  - Livedoor HOME
  - Peachy




## データの読み込み


In [1]:
import requests
import os
import tarfile

url = 'http://www.rondhuit.com/download/ldcc-20140209.tar.gz'
res = requests.get(url,stream=True)
if res.status_code == 200:
    with open('ldcc-20140209.tar.gz', 'wb') as file:
            for chunk in res.iter_content(chunk_size=1024):
                file.write(chunk)


tf = tarfile.open('ldcc-20140209.tar.gz', 'r')
tf.extractall('testdata')


## テキストデータのベクトル化
- テキストデータを数値化する代表的な手法
  - BoW　
  - TF-IDF

### BoW
- Bag of Words の略。単純にテキストデータに含まれる単語の数を数える手法。単純な手法なのでプログラムは組みやすい。
- scikit-learnではBoWを実現するためのモジュールが提供されていおり、簡単に試せる。
- 例「This is a pen. I have a pen.」

　　　　<img src="fig/bow.png" width="20%">

### BoW の実装
- Scikit-learnのCountVectorizerを使用する。
- 文章のデータ（リスト）を用意する。
- fit_transform() メソッドで単語を数える。
- fit_transform()の結果は単語を数えた結果の２次元配列が返される。
- get_feature_names（）で出現単語の一覧を確認できる。


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
docs = [ 'The sun is shining',
         'It is raining hard',
         'Do not go out in the rain']

bag = count_vec.fit_transform(docs)

print(count_vec.get_feature_names_out())

print(bag.toarray())


['do' 'go' 'hard' 'in' 'is' 'it' 'not' 'out' 'rain' 'raining' 'shining'
 'sun' 'the']
[[0 0 0 0 1 0 0 0 0 0 1 1 1]
 [0 0 1 0 1 1 0 0 0 1 0 0 0]
 [1 1 0 1 0 0 1 1 1 0 0 0 1]]


### 日本語の特徴
- 英語は単語と単語の間にスペース（空白）が入るので単語の分解が容易。日本語は単語と単語の間にスペースがない言語である。
- 単語に分解することを「分かち書き」と言う。日本語はどうように「分かち書き」をするかが重要である。
- 日本語は「分かち書き」するために文章を意味のある単語(形態素）まで分解する技術が必要。この技術を「形態素解析」という。
- 形態素解析のアルゴリズムは色々研究されているが、日本語のあいまいさにより完璧に「分かち書き」するのはとても難しい。
- 例えば次の文章は2通りの解釈ができる。

　「かれのくるまでまつ」

　「かれ（彼）　の　くる(来る）まで　まつ（待つ）」

　「かれ（彼）　の　くるま（車）　で　待つ（待つ）」

### Python形態素解析ライブラリ
- Pythonでは日本語形態素解析用のライブラリは「MeCab」と「janome」が有名である。
  - 「MeCab」はインストールに手間が掛かる。
  - 「janome」はインストールが簡単で容易に試せるので、「janome」を利用。

- テキストデータのクレンジング
  - 名詞のみを抽出対象とする
  - 出現頻度の低い単語、逆に出現頻度が多すぎる単語は除外する。
  - ベクトル化するコードの実行は非常に時間がかかる。



### ベクトル化の実装
#### ラベル

In [5]:
LABEL_NAME = ['dokujo-tsushin', 'it-life-hack','kaden-channel', 
             'livedoor-homme', 'movie-enter', 'peachy','smax',
             'sports-watch', 'topic-news']

#### テキストデータの読み込み

In [7]:
#データを読む
import os

news_dir='testdata/text'
excluded = ['CHANGES.txt','README.txt','LICENSE.txt']
docs ,labels, = [],[]
fnames = []

for label_no, label in enumerate(LABEL_NAME):
    dir = news_dir + '/' + label
    files = os.listdir(dir)
    for f in files:
        if f in excluded:
            continue
        fname = dir + '/' + f
        if not os.path.isfile(fname):
            continue
        with open(fname,'r') as f:
            try:
                data = ''.join(f.readlines()[2:])  # 1〜2行目は捨てる
                docs.append(data)
                labels.append(label_no)
                fnames.append(fname)
            except KeyError:
                pass


#### 読み込んだデータをPandasで扱う

In [ ]:
#Pandasで管理
import pandas as pd
import numpy as np
df = pd.DataFrame([labels,docs,fnames])
df = df.T
#data = df.reindex(np.random.permutation(df.index)).reset_index(drop=True)
data = df
data.columns=['L','T','F']
data.loc[ data['L'] == 8]

In [10]:
labels = np.array(data['L'])
docs_r = np.array(data['T'])
file_names = np.array(data['F'])

#### テキストデータをベクタライズ（参考）
　処理に時間がかかります。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from janome.tokenizer import Tokenizer

def extract_words(text):
    t = Tokenizer()
    token = t.tokenize(text)
    words = []
    for word in token:
        if word.part_of_speech.find('名詞') >= 0 :
            if not word.surface.isdigit():
                words.append(word.surface)
        
    return words

vectorizer = CountVectorizer(analyzer=extract_words,min_df=20,max_df=0.3)
bag = vectorizer.fit_transform(docs_r[:])
X = bag.toarray()
y_t = labels[:]
print(vectorizer.get_feature_names_out())

#### ベクタライズしたデータの保存（参考）


In [8]:
#保存
import numpy as np

np.savez('vectorize_result_all3',X)
np.save('labels3',y_t)
np.save('fnames3', file_names)

#### ベクタライズした結果をロードする

In [2]:
import numpy as np
Z = np.load('vectorize_result_all3.npz',allow_pickle=True)
X = Z['arr_0']
print(len(X[0]))
labels = np.load('labels3.npy',allow_pickle=True)
fnames = np.load('fnames3.npy',allow_pickle=True)

7020


## 次元削減
### 次元削減とは
学習データの次元数が多いと組み合わせが膨大になり機械学習の計算が効率良く行えなくなる。

これを「次元の呪い」と呼ばれている。

データの特徴を損なわずに次元を削減する手法を次元削減と言う。

### 次元削減の主な手法
- 主成分分析(PCA)
- 特異値分解(SVD)

いずれもscikit-learnでサポートされているアルゴリズム。今回は主成分分析を使う。



### 7020次元を300次元に次元削減

In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X = pca.fit_transform(X)

### 正解ラベルのOneHoneベクトル化

In [11]:
train_t = labels.reshape([len(X), 1])

# 正解ラベルの　OneHot化
from sklearn.preprocessing import OneHotEncoder

# encode label
# encoder = OneHotEncoder(n_values=max(train_t)+1)
encoder = OneHotEncoder()

train_y= encoder.fit_transform(train_t).toarray()
print(train_y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


## 計算モデル
　以下のようなモデルを考える。

　　　　<img src="fig/text_model.png" width="50%">

　　　

### Kerasモデル定義

(Kerasでの実装は演習にて実施します)

In [9]:
type(G)

numpy.ndarray